In [2]:
import xlrd
import pandas as pd
import xlsxwriter
import glob
import os
import re
import numpy as np
import sys

In [ ]:
xlsx_files = glob.glob('C:/Users/Rocku/Downloads/SeekingAlpha_EarningCalls/1-80000/*.xlsx')
len(xlsx_files)
# xlsx_files[:10]

In [ ]:
def save_error_urls(error_data, error_log):
    path = 'error_logs.txt'
        
    with open(path, 'a') as f:
        f.write(error_data + ',' + error_log + '\n')

In [ ]:
# sample_data = ['C:/Users/Rocku/Downloads/SeekingAlpha_EarningCalls/1-80000\\SeekingAlpha_1418571.xlsx']

In [ ]:
# for i, path in enumerate(sample_data):
success = 0
fail = 0

for i, path in enumerate(xlsx_files):
    if (i % 1000 == 0):
        print(i, 'success:', success, 'fail:', fail)

    try:
        data = pd.read_excel(path, header=None, index_col=0, encoding='utf-8')
        regex1 = re.compile('\s\W\s')

        if 'Executives' in data.index and (data.loc[['Executives'],1].count() != 0):
            data.loc['Executives',1] = data.loc['Executives',1].replace('&', 'and').replace('@', 'at')
            if (len(data.loc[['Executives'],1]) == 1):
                if (len(regex1.split(data.loc['Executives',1])) > 1):
                    data.loc['Executives',2] =  regex1.split(data.loc['Executives',1])[1]
                data.loc['Executives',1] = regex1.split(data.loc['Executives',1])[0]
            else:
                data.loc['Executives',2] = data.loc['Executives',1].str.split(r'\s\W\s').str[1]
                data.loc['Executives',1] = data.loc['Executives',1].str.split(r'\s\W\s').str[0]

        if ('Analysts' in data.index) and (data.loc[['Analysts'],1].count() != 0):
            data.loc['Analysts',1] = data.loc['Analysts',1].replace('&', 'and').replace('@', 'at')
            if (len(data.loc[['Analysts'],1]) == 1):
                if (len(regex1.split(data.loc['Analysts',1])) > 1):
                    data.loc['Analysts',2] =  regex1.split(data.loc['Analysts',1])[1]
                data.loc['Analysts',1] =  regex1.split(data.loc['Analysts',1])[0]
            else:
                data.loc['Analysts',2] = data.loc['Analysts',1].str.split(r'\s\W\s').str[1]
                data.loc['Analysts',1] = data.loc['Analysts',1].str.split(r'\s\W\s').str[0]

        columns = ['FileName', 'CompanyName', 'Exchange', 'Ticker', 'CallDate', 'UploadDate', 'Role', 'Position', 'Name']
        df = pd.DataFrame(columns=columns)

        FileName = path.split('\\')[-1]
        UploadDate = data.loc['UploadDate',1]
        CompanyName = data.loc['CompanyName',1]
        Exchange = data.loc['Exchange',1]
        Ticker = data.loc['Ticker',1]
        Subtitle = data.loc['Subtitle',1]
        CallDate = data.loc['CallDate',1]

        ex = data.loc[['Executives', 'Analysts'], :]

        for i in ex.iterrows():
            data_dict = {}
            data_dict['FileName'] = FileName
            data_dict['UploadDate'] = UploadDate
            data_dict['CompanyName'] = CompanyName
            data_dict['Exchange'] = Exchange
            data_dict['Ticker'] = Ticker
            data_dict['CallDate'] = CallDate
            data_dict['Role'] = i[0]
            data_dict['Name'] = i[1][1]
            data_dict['Position'] = i[1][2]

            regex = re.compile('\s+')
            if (len(data_dict['Exchange'].split(' ')) != 1):
                data_dict['Exchange'] = data_dict['Exchange'] + ':' + data_dict['Ticker']
                data_dict['Ticker'] = regex.split(data_dict['Exchange'])[0].replace(')', '')
                data_dict['CallDate'] = ' '.join(regex.split(data_dict['Exchange'])[-6:])
                data_dict['Exchange'] = None

            if (pd.isnull(data_dict['Name']) == True):
                pass
            else: 
                df1 = pd.DataFrame([data_dict], columns=columns)
                df = df.append(df1)

        if not os.path.isfile('index_file.csv'):
            df.to_csv('index_file.csv', header='column_names', index=False, encoding='utf-8')
        else:
            df.to_csv('index_file.csv', mode='a', header=False, index=False, encoding='utf-8')
        
        
        success += 1
        
    except:
        e = sys.exc_info()[1]
        save_error_urls(path.split('\\')[-1], str(e))
        fail += 1
        
        
print('------------end--------------')

# Remove the error data

In [ ]:
path = 'C:/Users/Rocku/Dropbox/Data_analysis/seekingalpha/error_logs_upto80000.txt'

bad_files = data = pd.read_table(path, header=-1, sep=',', encoding='utf-8')

In [ ]:
bad_files.iloc[:,0] =  'C:/Users/Rocku/Downloads/SeekingAlpha_EarningCalls/1-80000\\' + bad_files.iloc[:,0]
bad_file = bad_files.iloc[:,0].tolist()
len(bad_file)

In [ ]:
for raw_file in xlsx_files:
    if raw_file in bad_file:
        os.remove(raw_file)

In [ ]:
path1 = 'C:/Users/Rocku/Dropbox/Data_analysis/seekingalpha/index_file_1.csv'
path2 = 'C:/Users/Rocku/Dropbox/Data_analysis/seekingalpha/index_file_2.csv'

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

In [ ]:
df1.count()

In [ ]:
df2.count()

In [ ]:
df1.loc[pd.isnull(df1['CompanyName']), ['Ticker']] = "TCKRF"
df1.loc[pd.isnull(df1['CompanyName']), ['Exchange']] ="OTC"
df1.loc[pd.isnull(df1['CompanyName']), ['CompanyName']] = "Teck Resources Ltd."

In [ ]:
df2[pd.isnull(df2['CompanyName'])].count()

In [ ]:
df2[~df2['﻿FileName'].str.startswith('Seeking')]

In [ ]:
error_data = df2.loc[pd.isnull(df2['CompanyName']), '﻿FileName']

In [ ]:
error_data_list = list(set(error_data[error_data.str.startswith('Seeking')]))

In [ ]:
len(error_data_list)
error_data_list

In [ ]:
df2 = df2[df2['﻿FileName'].str.startswith('Seeking')]

In [ ]:
df2 = df2[pd.notnull(df2['CompanyName'])]

In [ ]:
df2 = df2[pd.notnull(df2['Name'])]

In [ ]:
df2.count()

In [ ]:
for raw_file in error_data_list:
    try:
        os.remove('C:/Users/Rocku/Downloads/SeekingAlpha_EarningCalls/1-80000\\'+raw_file)
    except:
        print(raw_file)

# Error correction #2 - Ticker contains lots of infomation --> split

In [93]:
del df1, df2

In [2]:
path1 = 'C:/Users/Rocku/Dropbox/Data_analysis/seekingalpha/index_file_1_revised.csv'
path2 = 'C:/Users/Rocku/Dropbox/Data_analysis/seekingalpha/index_file_2_revised.csv'

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

In [6]:
df1.count()

FileName       613940
CompanyName    613940
Exchange       591234
Ticker         613940
CallDate       611077
UploadDate     613940
Role           613940
Position       605875
Name           613940
dtype: int64

In [7]:
df2.count()

﻿FileName      485163
CompanyName    485163
Exchange       424010
Ticker         485146
CallDate       483393
UploadDate     485163
Role           485163
Position       468545
Name           485163
dtype: int64

In [8]:
df1[pd.notnull(df1['Ticker'].str.split(' ').str[1])].count()

FileName       52343
CompanyName    52343
Exchange       52343
Ticker         52343
CallDate       49482
UploadDate     52343
Role           52343
Position       52062
Name           52343
dtype: int64

In [9]:
df1[pd.notnull(df1['Ticker'].str.split('\s').str[1])]

,FileName,CompanyName,Exchange,Ticker,CallDate,UploadDate,Role,Position,Name
714,SeekingAlpha_1397181.xlsx,"Canfor Pulp Products, Inc.",OTC,"CFPUF Q1 2013 Earnings Call May 2, 2013 11",Donald B. Kayne – Chief Executive Officer,2013-05-02 21:15:04 -0400,Executives,Chief Executive Officer,Donald B. Kayne
715,SeekingAlpha_1397181.xlsx,"Canfor Pulp Products, Inc.",OTC,"CFPUF Q1 2013 Earnings Call May 2, 2013 11",Donald B. Kayne – Chief Executive Officer,2013-05-02 21:15:04 -0400,Executives,Chief Financial Officer,Alan R. Nicholl
716,SeekingAlpha_1397181.xlsx,"Canfor Pulp Products, Inc.",OTC,"CFPUF Q1 2013 Earnings Call May 2, 2013 11",Donald B. Kayne – Chief Executive Officer,2013-05-02 21:15:04 -0400,Executives,President,Brett R. Robinson
717,SeekingAlpha_1397181.xlsx,"Canfor Pulp Products, Inc.",OTC,"CFPUF Q1 2013 Earnings Call May 2, 2013 11",Donald B. Kayne – Chief Executive Officer,2013-05-02 21:15:04 -0400,Executives,Vice President of Sales and Marketing,Sean Curran
718,SeekingAlpha_1397181.xlsx,"Canfor Pulp Products, Inc.",OTC,"CFPUF Q1 2013 Earnings Call May 2, 2013 11",Donald B. Kayne – Chief Executive Officer,2013-05-02 21:15:04 -0400,Analysts,Raymond James Ltd.,Daryl Swetlishoff
719,SeekingAlpha_1397181.xlsx,"Canfor Pulp Products, Inc.",OTC,"CFPUF Q1 2013 Earnings Call May 2, 2013 11",Donald B. Kayne – Chief Executive Officer,2013-05-02 21:15:04 -0400,Analysts,TD Securities,Sean Steuart
720,SeekingAlpha_1397181.xlsx,"Canfor Pulp Products, Inc.",OTC,"CFPUF Q1 2013 Earnings Call May 2, 2013 11",Donald B. Kayne – Chief Executive Officer,2013-05-02 21:15:04 -0400,Analysts,"CIBC World Markets, Inc.",Mark Kennedy
721,SeekingAlpha_1397181.xlsx,"Canfor Pulp Products, Inc.",OTC,"CFPUF Q1 2013 Earnings Call May 2, 2013 11",Donald B. Kayne – Chief Executive Officer,2013-05-02 21:15:04 -0400,Analysts,RBC Capital Markets LLC,Paul Quinn
762,SeekingAlpha_1397241.xlsx,BG Group,OTCQX,"BRGYY Q1 2013 Earnings Call May 2, 2013 7","Chris Finlayson - Chief Executive Officer, Man...",2013-05-02 21:20:07 -0400,Executives,"Chief Executive Officer, Managing Director of ...",Chris Finlayson
763,SeekingAlpha_1397241.xlsx,BG Group,OTCQX,"BRGYY Q1 2013 Earnings Call May 2, 2013 7","Chris Finlayson - Chief Executive Officer, Man...",2013-05-02 21:20:07 -0400,Executives,"Interim Chief Financial Officer, Financial Con...",Den Jones


In [10]:
df1.loc[pd.notnull(df1['Ticker'].str.split(' ').str[1]), 'CallDate'] = \
df1.loc[pd.notnull(df1['Ticker'].str.split(' ').str[1]), 'Ticker'].str.split(r'\s+').str[-4:].str.join(' ')

df1.loc[pd.notnull(df1['Ticker'].str.split(' ').str[1]), 'Ticker'] = \
df1.loc[pd.notnull(df1['Ticker'].str.split(' ').str[1]), 'Ticker'].str.split(r'\s+').str[0]

In [11]:
df1[pd.notnull(df1['Ticker'].str.split(' ').str[1])].count()

FileName       0
CompanyName    0
Exchange       0
Ticker         0
CallDate       0
UploadDate     0
Role           0
Position       0
Name           0
dtype: int64

In [12]:
df2[pd.notnull(df2['Ticker'].str.split(' ').str[2])].count()

﻿FileName      8571
CompanyName    8571
Exchange       8571
Ticker         8571
CallDate       6827
UploadDate     8571
Role           8571
Position       8273
Name           8571
dtype: int64

In [13]:
df2[pd.notnull(df2['Ticker'].str.split(' ').str[2])]

,﻿FileName,CompanyName,Exchange,Ticker,CallDate,UploadDate,Role,Position,Name
1328,SeekingAlpha_1003571.xlsx,Petrobank Energy and Resources Ltd.,OTCPK,"PBEGF Q3 2012 Earnings Call November 13, 2012 10","John D. Wright ??President, Chief Executive Of...",2012-11-13 13:35,Executives,"President, Chief Executive Officer and Chairman",John D. Wright
1329,SeekingAlpha_1003571.xlsx,Petrobank Energy and Resources Ltd.,OTCPK,"PBEGF Q3 2012 Earnings Call November 13, 2012 10","John D. Wright ??President, Chief Executive Of...",2012-11-13 13:35,Executives,Vice President Finance and Chief Financial Off...,Peter Cheung
1330,SeekingAlpha_1003571.xlsx,Petrobank Energy and Resources Ltd.,OTCPK,"PBEGF Q3 2012 Earnings Call November 13, 2012 10","John D. Wright ??President, Chief Executive Of...",2012-11-13 13:35,Executives,Senior Vice President and Chief Operating Offi...,Chris J. Bloomer
1331,SeekingAlpha_1003571.xlsx,Petrobank Energy and Resources Ltd.,OTCPK,"PBEGF Q3 2012 Earnings Call November 13, 2012 10","John D. Wright ??President, Chief Executive Of...",2012-11-13 13:35,Executives,Senior Vice President and Chief Financial Offi...,Peter D. Scott
1332,SeekingAlpha_1003571.xlsx,Petrobank Energy and Resources Ltd.,OTCPK,"PBEGF Q3 2012 Earnings Call November 13, 2012 10","John D. Wright ??President, Chief Executive Of...",2012-11-13 13:35,Executives,General Counsel of Petrobank,Tim Sweeney
1573,SeekingAlpha_1004401.xlsx,Aurcana Corporation,OTCPK,"AUNFF Q3 2012 Earnings Call November 13, 2012 2","Catalin Chiloflischi - Manager, Investor Commu...",2012-11-13 16:21,Executives,"Manager, Investor Communications",Catalin Chiloflischi
1574,SeekingAlpha_1004401.xlsx,Aurcana Corporation,OTCPK,"AUNFF Q3 2012 Earnings Call November 13, 2012 2","Catalin Chiloflischi - Manager, Investor Commu...",2012-11-13 16:21,Executives,President,Lenic Rodriguez
1575,SeekingAlpha_1004401.xlsx,Aurcana Corporation,OTCPK,"AUNFF Q3 2012 Earnings Call November 13, 2012 2","Catalin Chiloflischi - Manager, Investor Commu...",2012-11-13 16:21,Analysts,Stonecap Securities,Christos Doulis
1612,SeekingAlpha_1004631.xlsx,Vivendi,OTC,"VIVDY Q3 2012 Earnings Call November 13, 2012 12",Jean-Michel Bonamy - Executive Vice President ...,2012-11-13 17:00,Executives,Executive Vice President of Investor Relations,Jean-Michel Bonamy
1613,SeekingAlpha_1004631.xlsx,Vivendi,OTC,"VIVDY Q3 2012 Earnings Call November 13, 2012 12",Jean-Michel Bonamy - Executive Vice President ...,2012-11-13 17:00,Executives,Chief Financial Officer and Member of the Mana...,Philippe G. H. Capron


In [14]:
df2.loc[pd.notnull(df2['Ticker'].str.split(' ').str[2]), 'CallDate'] = \
df2.loc[pd.notnull(df2['Ticker'].str.split(' ').str[2]), 'Ticker'].str.split(r'\s+').str[-4:].str.join(' ')

df2.loc[pd.notnull(df2['Ticker'].str.split(' ').str[2]), 'Ticker'] = \
df2.loc[pd.notnull(df2['Ticker'].str.split(' ').str[2]), 'Ticker'].str.split(r'\s+').str[0]

In [15]:
df2[pd.notnull(df2['Ticker'].str.split(' ').str[2])].count()

﻿FileName      0
CompanyName    0
Exchange       0
Ticker         0
CallDate       0
UploadDate     0
Role           0
Position       0
Name           0
dtype: int64

In [16]:
df1.to_csv('index_file_1_revised2.csv', header='column_names', index=False, encoding='utf-8')

In [17]:
df2.to_csv('index_file_2_revised2.csv', header='column_names', index=False, encoding='utf-8')

In [18]:
del df1, df2

# Concatanate two index files

In [19]:
path1 = 'C:/Users/Rocku/Dropbox/Data_analysis/seekingalpha/index_file_1_revised2.csv'
path2 = 'C:/Users/Rocku/Dropbox/Data_analysis/seekingalpha/index_file_2_revised2.csv'

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

In [20]:
df2.rename(columns={'\ufeffFileName': 'FileName'}, inplace=True)

In [21]:
df = pd.concat([df1,df2])

In [ ]:
df.to_csv('index_file.csv', header='column_names', index=False, encoding='utf-8')

# Data Cleaning #2 - remove 'None' Ticker file

In [390]:
path1 = 'C:/Users/Rocku/Dropbox/Data_analysis/seekingalpha/index_file.csv'
df1 = pd.read_csv(path1)
df1.count()

FileName       1099103
CompanyName    1099103
Exchange       1015244
Ticker         1099059
CallDate       1099075
UploadDate     1099103
Role           1099103
Position       1074420
Name           1099103
dtype: int64

In [391]:
df1[df1['Ticker'] == 'None'].count()

FileName       15362
CompanyName    15362
Exchange       15362
Ticker         15362
CallDate       15343
UploadDate     15362
Role           15362
Position       15015
Name           15362
dtype: int64

In [392]:
df = df1[~(df1['Ticker'] == 'None')]
df.count()

FileName       1083741
CompanyName    1083741
Exchange        999882
Ticker         1083697
CallDate       1083732
UploadDate     1083741
Role           1083741
Position       1059405
Name           1083741
dtype: int64

In [393]:
df.to_csv('index_file2.csv', header='column_names', index=False, encoding='utf-8')

In [394]:
none_data_list = list(set(df1.loc[df1['Ticker'] == 'None', 'FileName']))
len(none_data)

1728

In [398]:
cant_remove = []

for raw_file in none_data_list:
    try:
        os.remove('C:/Users/Rocku/Downloads/SeekingAlpha_EarningCalls/raw_xlsx/' + raw_file)
    except:
        cant_remove.append(raw_file)
        
len(cant_remove)

0

# Divide two index files 

In [3]:
path1 = 'C:/Users/Rocku/Downloads/SeekingAlpha_EarningCalls/index_file.csv'

df1 = pd.read_csv(path1)

In [4]:
df1.count()

FileName       1083741
CompanyName    1083741
Exchange        999882
Ticker         1083697
CallDate       1083732
UploadDate     1083741
Role           1083741
Position       1059405
Name           1083741
dtype: int64

In [15]:
df1[:600000]

,FileName,CompanyName,Exchange,Ticker,CallDate,UploadDate,Role,Position,Name
0,SeekingAlpha_1396301.xlsx,Magellan Midstream Partners LP,NYSE,MMP,"May 02, 2013 1:30 pm ET",2013-05-02 18:00:07 -0400,Executives,"Chairman of Magellan GP LLC, Chief Executive O...",Michael N. Mears
1,SeekingAlpha_1396301.xlsx,Magellan Midstream Partners LP,NYSE,MMP,"May 02, 2013 1:30 pm ET",2013-05-02 18:00:07 -0400,Executives,"Chief Financial Officer of Magellan Gp Llc, Pr...",John D. Chandler
2,SeekingAlpha_1396301.xlsx,Magellan Midstream Partners LP,NYSE,MMP,"May 02, 2013 1:30 pm ET",2013-05-02 18:00:07 -0400,Analysts,NaN,Edward Rowe
3,SeekingAlpha_1396301.xlsx,Magellan Midstream Partners LP,NYSE,MMP,"May 02, 2013 1:30 pm ET",2013-05-02 18:00:07 -0400,Analysts,"Barclays Capital, Research Division",Brian J. Zarahn
4,SeekingAlpha_1396301.xlsx,Magellan Midstream Partners LP,NYSE,MMP,"May 02, 2013 1:30 pm ET",2013-05-02 18:00:07 -0400,Analysts,"Goldman Sachs Group Inc., Research Division",Steven C. Sherowski
5,SeekingAlpha_1396301.xlsx,Magellan Midstream Partners LP,NYSE,MMP,"May 02, 2013 1:30 pm ET",2013-05-02 18:00:07 -0400,Analysts,"Wells Fargo Securities, LLC, Research Division",Sharon Lui
6,SeekingAlpha_1396301.xlsx,Magellan Midstream Partners LP,NYSE,MMP,"May 02, 2013 1:30 pm ET",2013-05-02 18:00:07 -0400,Analysts,"Crédit Suisse AG, Research Division",John Edwards
7,SeekingAlpha_1396301.xlsx,Magellan Midstream Partners LP,NYSE,MMP,"May 02, 2013 1:30 pm ET",2013-05-02 18:00:07 -0400,Analysts,NaN,James Jampel
8,SeekingAlpha_1396301.xlsx,Magellan Midstream Partners LP,NYSE,MMP,"May 02, 2013 1:30 pm ET",2013-05-02 18:00:07 -0400,Analysts,"Morningstar Inc., Research Division",Connie Hsu
9,SeekingAlpha_1396301.xlsx,Magellan Midstream Partners LP,NYSE,MMP,"May 02, 2013 1:30 pm ET",2013-05-02 18:00:07 -0400,Analysts,NaN,Norman Kramer


In [18]:
df_first = df1[:599984]
df_second = df1[599984:]

In [21]:
df_first.tail()
# df_second.head()

,FileName,CompanyName,Exchange,Ticker,CallDate,UploadDate,Role,Position,Name
599979,SeekingAlpha_4076572.xlsx,Toro Co,NYSE,TTC,"May 25, 2017 11:00 AM ET",2017-05-25 14:59:07 -0400,Analysts,Dougherty and Company,Jon Fisher
599980,SeekingAlpha_4076572.xlsx,Toro Co,NYSE,TTC,"May 25, 2017 11:00 AM ET",2017-05-25 14:59:07 -0400,Analysts,Seaport Global,Mike Shlisky
599981,SeekingAlpha_4076572.xlsx,Toro Co,NYSE,TTC,"May 25, 2017 11:00 AM ET",2017-05-25 14:59:07 -0400,Analysts,Cleveland Research,Tom Mahoney
599982,SeekingAlpha_4076572.xlsx,Toro Co,NYSE,TTC,"May 25, 2017 11:00 AM ET",2017-05-25 14:59:07 -0400,Analysts,Longbow Research,David MacGregor
599983,SeekingAlpha_4076572.xlsx,Toro Co,NYSE,TTC,"May 25, 2017 11:00 AM ET",2017-05-25 14:59:07 -0400,Analysts,Sidoti,Joe Mondillo


In [22]:
df_first.to_csv('index_file_p1.csv', header='column_names', index=False, encoding='utf-8')
df_second.to_csv('index_file_p2.csv', header='column_names', index=False, encoding='utf-8')